<h1 style='color: #c52b10; font-size: 34px; font-weight: bold;'>Sistema de Recomendação de Filmes

</h1>


# <font color="#10c586" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #10c586;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b> Desenvolver um sistema de recomendação de filmes com base em características como gênero, palavras-chave, popularidade e classificação média, etc. </b></i>     
</p>

---

# <font color="#10c586" style="font-size: 30px;">Roteiro</font>
<hr style="border: 2px solid #10c586;">

<h1 style='color: black; font-size:22px; font-weight: bold;'> 1. Pré-processamento de dados
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'>2. Criação da matriz de características

</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
3. Cálculo da similaridade entre filmes
</h1>

---

<h1 style='color: black; font-size: 22px; font-weight: bold;'> 
4. Implementação do sistema de recomendação
</h1>

---


# <font color="#10c586" style="font-size: 30px;">1. Pré-processamento de dados</font>
<hr style="border: 2px solid #10c586;">

In [346]:
# IMPORTANDO BIBLIOTECAS

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [330]:
# IMPORTANDO BASE DE DADOS

df = pd.read_csv('tmdb_5000_movies.csv')

df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [331]:
# RENOMEANDO AS COLUNAS

traducao = {
    'budget': 'Orcamento',
    'genres': 'Gêneros',
    'homepage': 'Página inicial',
    'id': 'ID',
    'keywords': 'Palavras-chave',
    'original_language': 'Idioma original',
    'original_title': 'Título original',
    'overview': 'Visão geral',
    'popularity': 'Popularidade',
    'production_companies': 'Empresas de produção',
    'production_countries': 'Países de produção',
    'release_date': 'Data de lançamento',
    'revenue': 'Receita',
    'runtime': 'Duração',
    'spoken_languages': 'Idiomas falados',
    'status': 'Status',
    'tagline': 'Slogan',
    'title': 'Título',
    'vote_average': 'Media_de_votos',
    'vote_count': 'Contagem_de_votos'
}

df = df.rename(columns=traducao)
df.head(2)

,Orcamento,Gêneros,Página inicial,ID,Palavras-chave,Idioma original,Título original,Visão geral,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Status,Slogan,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


---
## <font color=#c5106e style="font-size: 30px;">Transformar coluna de Data de lançamento em datetime</font>

---

In [332]:
# DANDO UMA OLHADA NA COLUNA DE LANÇAMENTOS

display(type(df['Data de lançamento'][0]))

#Converter representação de data em formato de string para um objeto de data no Python
 
from datetime import datetime

df['Data de lançamento'] = pd.to_datetime(df['Data de lançamento']) 

display(type(df['Data de lançamento'][0]))

str

pandas._libs.tslibs.timestamps.Timestamp

```python
# Transforma a lista de listas em uma lista simples
lista_simples = [item for sublist in lista_generos_filmes for item in sublist]

# Obtém os valores únicos da lista
valores_unicos = set(lista_simples)

df['Gêneros'] = lista_simples[:4803]
```

---
## <font color=#c5106e style="font-size: 30px;">Removendo Dados inconsistentes</font>

---

In [333]:
df.describe()

,Orcamento,ID,Popularidade,Receita,Duração,Media_de_votos,Contagem_de_votos
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


---
## <font color=#c5106e style="font-size: 30px;">Não vamos querer nenhum filme com:</font>
---

*  <font color=#c5106e style="font-size: 28px;">Orçamento Zero</font>

---

* <font color=#c5106e style="font-size: 28px;">Receita Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Duração Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Média de votos Zero</font>
---

* <font color=#c5106e style="font-size: 28px;">Contagem de votos Zero</font>

---

In [334]:
df = df.query('Orcamento > 0 and Receita > 0 and Duração > 0 and Media_de_votos > 0 and Contagem_de_votos > 0')

---
## <font color=#c5106e style="font-size: 30px;">Mais de 75% dos filmes tem mais de 100 votos. Vamos filtrar</font>
---

In [335]:
df = df.query('Contagem_de_votos >= 100') # para assim só termos filmes com estatísticas relevantes de nota 

In [336]:
df.drop(['Página inicial', 'ID', 'Visão geral', 'Slogan', 'Status', 'Palavras-chave'], axis=1, inplace=True)
df

,Orcamento,Gêneros,Idioma original,Título original,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,Avatar,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",en,Pirates of the Caribbean: At World's End,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,Spectre,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",en,The Dark Knight Rises,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",en,John Carter,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,4000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",en,The Signal,27.662696,"[{""name"": ""Automatik Entertainment"", ""id"": 281...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2014-03-15,600896,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Signal,5.8,631
4773,27000,"[{""id"": 35, ""name"": ""Comedy""}]",en,Clerks,19.748658,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1994-09-13,3151130,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Clerks,7.4,755
4788,12000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 35, ""nam...",en,Pink Flamingos,4.553644,"[{""name"": ""Dreamland Productions"", ""id"": 407}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1972-03-12,6000000,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Pink Flamingos,6.2,110
4796,7000,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",en,Primer,23.307949,"[{""name"": ""Thinkfilm"", ""id"": 446}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2004-10-08,424760,77.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Primer,6.9,658


In [337]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2715 entries, 0 to 4798
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Orcamento             2715 non-null   int64         
 1   Gêneros               2715 non-null   object        
 2   Idioma original       2715 non-null   object        
 3   Título original       2715 non-null   object        
 4   Popularidade          2715 non-null   float64       
 5   Empresas de produção  2715 non-null   object        
 6   Países de produção    2715 non-null   object        
 7   Data de lançamento    2715 non-null   datetime64[ns]
 8   Receita               2715 non-null   int64         
 9   Duração               2715 non-null   float64       
 10  Idiomas falados       2715 non-null   object        
 11  Título                2715 non-null   object        
 12  Media_de_votos        2715 non-null   float64       
 13  Contagem_de_votos 

---
## <font color=#c5106e style="font-size: 30px;">Conformar a coluna de Gêneros, Empresas de produção, Países de produção, Idiomas falados para a forma que queremos</font>

---

In [338]:
def colunas_para_lista(df, colunas):
    lista_resultante = []
    for coluna in colunas:
        lista_coluna = df[coluna]
        lista_coluna = list(lista_coluna)
        lista_resultante.append(lista_coluna)
    return lista_resultante

# Lista de colunas que desejamos transformar em listas
colunas_transformar = ['Gêneros', 'Empresas de produção', 'Países de produção', 'Idiomas falados']

import json


# Função para extrair os nomes dos gêneros de uma string em formato JSON
def extrair_dados(dados_str):
    dados_json = json.loads(dados_str)
    return [dados['name'] for dados in dados_json]

i = 0

for dados in colunas_para_lista(df, colunas_transformar):
    
    lista_dados_filmes = [extrair_dados(dados_str) for dados_str in dados]
    df[colunas_transformar[i]] = lista_dados_filmes
    i=i+1
df

,Orcamento,Gêneros,Idioma original,Título original,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Título,Media_de_votos,Contagem_de_votos
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",en,Avatar,150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Avatar,7.2,11800
1,300000000,"[Adventure, Fantasy, Action]",en,Pirates of the Caribbean: At World's End,139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[Action, Adventure, Crime]",en,Spectre,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Spectre,6.3,4466
3,250000000,"[Action, Crime, Drama, Thriller]",en,The Dark Knight Rises,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],The Dark Knight Rises,7.6,9106
4,260000000,"[Action, Adventure, Science Fiction]",en,John Carter,43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,4000000,"[Thriller, Science Fiction]",en,The Signal,27.662696,"[Automatik Entertainment, Low Spark Films, Sig...",[United States of America],2014-03-15,600896,95.0,[English],The Signal,5.8,631
4773,27000,[Comedy],en,Clerks,19.748658,"[Miramax Films, View Askew Productions]",[United States of America],1994-09-13,3151130,92.0,[English],Clerks,7.4,755
4788,12000,"[Horror, Comedy, Crime]",en,Pink Flamingos,4.553644,[Dreamland Productions],[United States of America],1972-03-12,6000000,93.0,[English],Pink Flamingos,6.2,110
4796,7000,"[Science Fiction, Drama, Thriller]",en,Primer,23.307949,[Thinkfilm],[United States of America],2004-10-08,424760,77.0,[English],Primer,6.9,658


---
## <font color=#c5106e style="font-size: 30px;">Idioma original: substituir siglas</font>

---

In [339]:
df['Idioma original'].unique()

dicionario_idiomas = {
    'en': 'Inglês',
    'ja': 'Japonês',
    'fr': 'Francês',
    'zh': 'Chinês',
    'ko': 'Coreano',
    'te': 'Telugu',
    'cn': 'Chinês',
    'es': 'Espanhol',
    'it': 'Italiano',
    'nl': 'Holandês',
    'ru': 'Russo',
    'de': 'Alemão',
    'th': 'Tailandês',
    'xx': 'Desconhecido',
    'da': 'Dinamarquês',
    'nb': 'Norueguês',
    'pt': 'Português',
    'pl': 'Polonês',
    'he': 'Hebraico',
    'hi': 'Hindi',
    'id': 'Indonésio',
    'ro': 'Romeno',
    'no': 'Norueguês',
    'fa': 'Persa'
}

df['Idioma original'] = df['Idioma original'].map(dicionario_idiomas)
df.columns

Index(['Orcamento', 'Gêneros', 'Idioma original', 'Título original',
       'Popularidade', 'Empresas de produção', 'Países de produção',
       'Data de lançamento', 'Receita', 'Duração', 'Idiomas falados', 'Título',
       'Media_de_votos', 'Contagem_de_votos'],
      dtype='object')

In [340]:
idiomas_contagem = df['Idioma original'].value_counts()
idiomas_frequentes = idiomas_contagem[idiomas_contagem > 8].index.tolist()
display(idiomas_contagem)
display(idiomas_frequentes)
idiomas_frequentes.append('Português')

# df filtrado
df = df[df['Idioma original'].isin(idiomas_frequentes)]

Inglês          2623
Francês           18
Espanhol          13
Chinês            12
Japonês           10
Alemão             8
Italiano           6
Coreano            4
Dinamarquês        3
Indonésio          2
Português          2
Norueguês          2
Tailandês          2
Russo              2
Desconhecido       1
Holandês           1
Polonês            1
Hebraico           1
Hindi              1
Telugu             1
Romeno             1
Persa              1
Name: Idioma original, dtype: int64

['Inglês', 'Francês', 'Espanhol', 'Chinês', 'Japonês']

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2678 entries, 0 to 4798
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Orcamento             2678 non-null   int64         
 1   Gêneros               2678 non-null   object        
 2   Idioma original       2678 non-null   object        
 3   Título original       2678 non-null   object        
 4   Popularidade          2678 non-null   float64       
 5   Empresas de produção  2678 non-null   object        
 6   Países de produção    2678 non-null   object        
 7   Data de lançamento    2678 non-null   datetime64[ns]
 8   Receita               2678 non-null   int64         
 9   Duração               2678 non-null   float64       
 10  Idiomas falados       2678 non-null   object        
 11  Título                2678 non-null   object        
 12  Media_de_votos        2678 non-null   float64       
 13  Contagem_de_votos 

---

# <font color="#10c586" style="font-size: 30px;">2. Criação da Matriz de Características</font>
<hr style="border: 2px solid #10c586;">

---
## <font color=#c5106e style="font-size: 30px;">codificação one-hot para variáveis categóricas</font>

---


## <font style="font-size: 25px;">A codificação one-hot é uma técnica usada para converter variáveis categóricas em uma representação binária, onde cada categoria é representada como uma coluna binária separada. Essa técnica é amplamente utilizada para lidar com variáveis categóricas em algoritmos de aprendizado de máquina, pois permite que o modelo entenda e utilize essas informações de forma adequada.</font>

---

In [342]:
colunas_codi_one_hot = ['Gêneros', 'Idioma original']
for cols in colunas_codi_one_hot:
    # Realizar a codificação one-hot para a coluna
    display(df[cols].apply(pd.Series))
    coluna_one_hot = pd.get_dummies(df[cols].apply(pd.Series).stack(), dtype='str').groupby(level=0).sum()

    # Concatenar as colunas codificadas de gêneros no DataFrame original
    df = pd.concat([df, coluna_one_hot], axis=1)

    # Remover a coluna original 
    df = df.drop(cols, axis=1)
    
# Substituir os espaços em branco por 0
df.replace("", 0, inplace=True)
df

0     0             Action
      1          Adventure
      2            Fantasy
      3    Science Fiction
1     0          Adventure
                ...       
4796  1              Drama
      2           Thriller
4798  0             Action
      1              Crime
      2           Thriller
Length: 7237, dtype: object

0     0      Inglês
1     0      Inglês
2     0      Inglês
3     0      Inglês
4     0      Inglês
             ...   
4758  0      Inglês
4773  0      Inglês
4788  0      Inglês
4796  0      Inglês
4798  0    Espanhol
Length: 2678, dtype: object

,Orcamento,Título original,Popularidade,Empresas de produção,Países de produção,Data de lançamento,Receita,Duração,Idiomas falados,Título,Media_de_votos,Contagem_de_votos,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Chinês,Espanhol,Francês,Inglês,Japonês,Português
0,237000000,Avatar,150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Avatar,7.2,11800,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,300000000,Pirates of the Caribbean: At World's End,139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Pirates of the Caribbean: At World's End,6.9,4500,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,245000000,Spectre,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",2015-10-26,880674609,148.0,"[Français, English, Español, Italiano, Deutsch]",Spectre,6.3,4466,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,250000000,The Dark Knight Rises,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],2012-07-16,1084939099,165.0,[English],The Dark Knight Rises,7.6,9106,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,260000000,John Carter,43.926995,[Walt Disney Pictures],[United States of America],2012-03-07,284139100,132.0,[English],John Carter,6.1,2124,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,4000000,The Signal,27.662696,"[Automatik Entertainment, Low Spark Films, Sig...",[United States of America],2014-03-15,600896,95.0,[English],The Signal,5.8,631,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
4773,27000,Clerks,19.748658,"[Miramax Films, View Askew Productions]",[United States of America],1994-09-13,3151130,92.0,[English],Clerks,7.4,755,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4788,12000,Pink Flamingos,4.553644,[Dreamland Productions],[United States of America],1972-03-12,6000000,93.0,[English],Pink Flamingos,6.2,110,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4796,7000,Primer,23.307949,[Thinkfilm],[United States of America],2004-10-08,424760,77.0,[English],Primer,6.9,658,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0


---
## <font color=#c5106e style="font-size: 30px;"> Escalonamento Min-Max para variáveis numéricas</font>

---

In [310]:
colunas_escala_min_max = ['Orcamento', 'Popularidade', 'Receita', 'Duração',  'Media_de_votos', 'Contagem_de_votos']

# Criando o objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajustando e transformando os dados
dados_escalonados = scaler.fit_transform(df[colunas_escala_min_max])
df[colunas_escala_min_max] = dados_escalonados   

In [311]:
cols_descartar = ['Empresas de produção','Países de produção', 'Data de lançamento', 'Idiomas falados', 'Título']
df.drop(cols_descartar, axis=1,inplace=True)
df.set_index('Título original', inplace=True)
df

,Orcamento,Popularidade,Receita,Duração,Media_de_votos,Contagem_de_votos,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western,Chinês,Espanhol,Francês,Inglês,Japonês,Português
Título original,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Avatar,0.623684,0.170828,1.000000,0.535135,0.763636,0.857017,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
Pirates of the Caribbean: At World's End,0.789474,0.157845,0.344696,0.572973,0.709091,0.322297,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Spectre,0.644737,0.121590,0.315884,0.459459,0.600000,0.319807,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
The Dark Knight Rises,0.657895,0.127235,0.389151,0.551351,0.836364,0.659684,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
John Carter,0.684211,0.049038,0.101916,0.372973,0.563636,0.148257,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Signal,0.010526,0.030441,0.000216,0.172973,0.509091,0.038895,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
Clerks,0.000071,0.021391,0.001130,0.156757,0.800000,0.047978,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Pink Flamingos,0.000032,0.004016,0.002152,0.162162,0.581818,0.000732,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


---
# <font color="#10c586" style="font-size: 30px;">3. Cálculo da similaridade entre filmes</font>
<hr style="border: 2px solid #10c586;">

---
## <font color=#c5106e style="font-size: 30px;"> Similaridade do Cosseno</font>

---

In [312]:
# Calcular a similaridade do cosseno entre os filmes
similaridade_cosseno_filmes = cosine_similarity(df)

# <font color="#10c586" style="font-size: 30px;">4. Implementação do Sistema de Recomendação</font>
<hr style="border: 2px solid #10c586;">


---
## <font color=#c5106e style="font-size: 30px;"> Sistema de Recomendação </font>  <font color=#c5106e style="font-size: 30px;">com</font> <u><font color=#c5106e style="font-size: 30px;">modelos baseados em conteúdo usando o cálculo de similaridade do cosseno</font></u>
---

In [313]:
def recomendar_filmes(df, filme_referencia, num_recomendacoes):

    # Verificar se o filme de referência está presente no DataFrame
    if filme_referencia not in df.index:
        print(f"O filme '{filme_referencia}' não foi encontrado no DataFrame.")
        return None

    # Obter o índice numérico do filme de referência
    indice_filme_referencia = df.index.get_loc(filme_referencia)

    # Obter os índices dos filmes mais similares ao filme de referência
    filmes_similares = similaridade_cosseno_filmes[indice_filme_referencia].argsort()[:-num_recomendacoes-1:-1]

    # Filmes mais similares
    filmes_recomendados = df.index[filmes_similares].to_frame()
    filmes_recomendados.reset_index(drop=True, inplace=True)
    
    return filmes_recomendados


# Exemplo de uso
filme_referencia = 'The Dark Knight'
num_recomendacoes = 6
filmes_recomendados = recomendar_filmes(df, filme_referencia, num_recomendacoes)

filmes_recomendados[1:]

,Título original
1,The Dark Knight Rises
2,Scarface
3,Drive
4,Man on Fire
5,Training Day


In [345]:
import pandas as pd

data = [10, 20, 30, 40]
series = pd.Series(data)


[10, 20, 30, 40]